In [2]:
from py4j.java_gateway import JavaGateway, JavaObject, GatewayParameters
from py4j.java_gateway import java_import, get_field

host="localhost"
port=2333
callback_port=2334
gateway = JavaGateway(python_proxy_port=callback_port,
                      gateway_parameters=GatewayParameters(address=host, port=port, auto_field=True))
random = gateway.jvm.java.util.Random()   # create a java.util.Random instance
number1 = random.nextInt(10)              # call the Random.nextInt method
number2 = random.nextInt(10)
print(number1, number2)

j = gateway.new_jvm_view()
java_import(j, 'com.hankcs.hanlp.*')
java_import(j, 'java.util.*')
java_import(j, 'com.hankcs.hanlp.util.*')
java_import(j, 'com.hankcs.hanlp.utility.*')
java_import(j, 'com.hankcs.hanlp.corpus.tag.Nature')
# stack=j.Stack()
# m = j.HashMap()

text="你好"
result=j.HanLP.convertToPinyinList(text)
for pinyin in result:
    print(pinyin)

6 3
ni3
hao3


In [3]:
pcNature = j.Nature.create("tech")
nature_c=j.Nature
natures=gateway.new_array(nature_c,1)
natures[0]=pcNature
j.LexiconUtility.setAttribute("苹果电脑", natures)
j.LexiconUtility.setAttribute("阿尔法狗", natures)
termList = j.HanLP.segment("苹果电脑可以运行开源阿尔法狗代码吗")
for term in termList:
    print(term, )

苹果电脑/tech
可以/v
运行/vn
开源/vn
阿尔法狗/tech
代码/n
吗/y


In [4]:
raw="苹果电脑可以运行开源阿尔法狗代码吗"
sentence = j.HanLP.parseDependency(raw)
for word in sentence.getWordArray():
# for word in sentence.getWordArrayWithRoot():
    print("%s --(%s)--> %s"%(word.LEMMA, word.DEPREL, word.HEAD.LEMMA))

苹果电脑 --(主谓关系)--> 运行
可以 --(状中结构)--> 运行
运行 --(核心关系)--> ##核心##
开源阿尔法狗 --(定中关系)--> 代码
代码 --(动宾关系)--> 运行
吗 --(右附加关系)--> 运行


In [5]:
def describe_rel(word, result):
    if word.DEPREL=="主谓关系":
        result.append("\tactor: {}".format(word.LEMMA))
    elif word.DEPREL=="动宾关系":
        result.append("\tobject: {}".format(word.LEMMA))
    elif word.DEPREL=="标点符号":
        pass
    else:    
        result.append("\trel.{}({}): {}".format(word.POSTAG, word.DEPREL, word.LEMMA))

def get_pinyin(sentence):
    pinyin_list = j.HanLP.convertToPinyinList(sentence)
    l=[]
    for pinyin in pinyin_list:
        l.append("%s" % pinyin.getPinyinWithToneMark())
    return (" ".join(l))

java_import(j, 'com.hankcs.hanlp.corpus.dependency.CoNll.CoNLLWord')
java_import(j, "com.hankcs.hanlp.tokenizer.NLPTokenizer")
def parse_tree(sentence):    
    conll = j.HanLP.parseDependency(sentence)
    coreindex=0
    result=[]
    for word in conll.iterator():
        if word.HEAD==j.CoNLLWord.ROOT:
            coreindex=word.ID
            result.append("core: {} - {}".format(word.POSTAG, word.LEMMA))
    for word in conll.iterator():
        if word.HEAD.ID==coreindex:
            describe_rel(word, result)

    result.append("⊕ "+str(j.NLPTokenizer.analyze(sentence)))
    result.append("⊙ "+str(j.NLPTokenizer.analyze(sentence).translateLabels()))
    result.append("ﺴ "+get_pinyin(sentence))
    result.append("☫ "+j.HanLP.convertToTraditionalChinese(sentence))
    result.append("% "+sentence)
    return '\n'.join(result)

raw="苹果电脑可以运行开源阿尔法狗代码吗"
result=parse_tree(raw)
print(result)

core: v - 运行
	actor: 苹果电脑
	rel.v(状中结构): 可以
	object: 代码
	rel.y(右附加关系): 吗
⊕ 苹果电脑/tech 可以/v 运行/v 开源阿尔法狗/v 代码/n 吗/y
⊙ 苹果电脑/tech 可以/动词 运行/动词 开源阿尔法狗/动词 代码/名词 吗/语气词
ﺴ píng guǒ diàn nǎo kě yǐ yùn xíng kāi yuán ā ěr fǎ gǒu dài mǎ ma
☫ 蘋果電腦可以運行開源阿爾法狗代碼嗎
% 苹果电脑可以运行开源阿尔法狗代码吗


In [17]:
raw="苹果电脑可以运行开源阿尔法狗代码吗"
result=parse_tree(raw)
print(result)

core: v - 运行
	actor: 苹果电脑
	rel.v(状中结构): 可以
	object: 代码
	rel.y(右附加关系): 吗
⊕ 苹果电脑/tech 可以/v 运行/v 开源阿尔法狗/v 代码/n 吗/y
ﺴ píng guǒ diàn nǎo kě yǐ yùn xíng kāi yuán ā ěr fǎ gǒu dài mǎ ma
☫ 蘋果電腦可以運行開源阿爾法狗代碼嗎
% 苹果电脑可以运行开源阿尔法狗代码吗


In [21]:
# 可以直接遍历子树，从某棵子树的某个节点一路遍历到虚根
sentence = j.HanLP.parseDependency(raw)
wordArray = sentence.getWordArray()
head = wordArray[3]
while head is not None:
    if head == j.CoNLLWord.ROOT:
        print(head.LEMMA)
    else:
        print("%s --(%s)--> "%(head.LEMMA, head.DEPREL))
    head = head.HEAD

开源阿尔法狗 --(定中关系)--> 
代码 --(动宾关系)--> 
运行 --(核心关系)--> 
##核心##


In [7]:
from tabulate import tabulate
table_header = ['a','rel','b']
table_data = []

# 顺序遍历
sentence = j.HanLP.parseDependency(raw)
wordArray = sentence.getWordArray()
for word in wordArray:
    # print("%s --(%s)--> %s"%(word.LEMMA, word.DEPREL, word.HEAD.LEMMA))
    table_data.append((word.LEMMA, word.DEPREL, word.HEAD.LEMMA))
print(tabulate(table_data, headers=table_header, tablefmt='psql'))    

+--------------+------------+----------+
| a            | rel        | b        |
|--------------+------------+----------|
| 苹果电脑     | 主谓关系   | 运行     |
| 可以         | 状中结构   | 运行     |
| 运行         | 核心关系   | ##核心## |
| 开源阿尔法狗 | 定中关系   | 代码     |
| 代码         | 动宾关系   | 运行     |
| 吗           | 右附加关系 | 运行     |
+--------------+------------+----------+


In [1]:
from sagas.ofbiz.resource_mappings import ResourceMappings
from protobuf_utils import write_proto_to, read_proto
data_file='./data/resources/form_res_sample.data'
forms=['component://party/widget/partymgr/LookupForms.xml;LookupPartyName;zh']
rm=ResourceMappings()
meta_package=rm.build_package(forms)
write_proto_to(meta_package, data_file)

get form component://party/widget/partymgr/LookupForms.xml;LookupPartyName;zh


In [1]:
from sagas.ofbiz.entities import OfEntity as e, oc, finder
form_loc='component://webtools/widget/ServiceForms.xml;@JobManagerLockEnable;zh_CN'
# form_loc='component://webtools/widget/ServiceForms.xml;JobManagerLockEnable;en_US'
# form_loc='component://webtools/widget/ServiceForms.xml;AddJobManagerLock;zh_CN'
forms=oc.component('form_mgr')
form=forms.getMetaForm(form_loc)

In [2]:
form_loc='component://content/widget/layout/LayoutForms.xml;LayoutSubContentMaster;zh_CN'
# form_loc='component://content/widget/layout/LayoutForms.xml;EditLayout;zh_CN'
form=forms.getMetaForm(form_loc)

In [4]:
from forms_pb2 import MetaForm, MetaMappingPackage, MetaFieldMapping, MetaFieldMappings, SUBMIT, RESET

meta_package=MetaMappingPackage()
read_proto(meta_package, data_file)
mapflds=meta_package.mappings['PartyLastName']
for mapfld in mapflds.fields:
    print(mapfld.fieldName, mapfld.fieldTitle)

lastName 姓氏


In [5]:
from sagas.ofbiz.forms import get_form_list, collect_forms
form_list = get_form_list()
form_index = collect_forms(form_list)

In [6]:
from sagas.ofbiz.resource_mappings import ResourceMappings
from protobuf_utils import write_proto_to, read_proto
from forms_pb2 import MetaForm, MetaMappingPackage, MetaFieldMapping, MetaFieldMappings, SUBMIT, RESET

print(len(form_index.items()))

forms=[]
for k,locs in form_index.items():
    for loc in locs:
        # loc.name, loc.location, loc.uri
        form_loc=loc.uri+';'+k+';zh_CN'
        forms.append(form_loc)

data_file='./data/resources/form_res.data'
rm=ResourceMappings()
meta_package=rm.build_package(forms)
write_proto_to(meta_package, data_file)
print('done.')

1802
get form component://common/widget/PortalPageForms.xml;ListPortalPages;zh_CN
get form component://common/widget/LookupForms.xml;ListPortalPages;zh_CN
get form component://common/widget/PortalPageForms.xml;NewPortalPage;zh_CN
get form component://common/widget/PortalPageForms.xml;PortletCategoryAndPortlet;zh_CN
get form component://common/widget/PortalPageForms.xml;PortletList;zh_CN
get form component://common/widget/PortalPageForms.xml;FindGenericEntity;zh_CN
get form component://common/widget/PortalPageForms.xml;EditPortalPageColumnWidth;zh_CN
get form component://common/widget/PortletEditForms.xml;CommonPortletEdit;zh_CN
get form component://common/widget/PortletEditForms.xml;GenericPortalPageParam;zh_CN
get form component://common/widget/PortletEditForms.xml;FindGenericEntityParam;zh_CN
get form component://common/widget/PortletEditForms.xml;GenericScreenletParam;zh_CN
get form component://common/widget/PortletEditForms.xml;GenericScreenletAjaxParam;zh_CN
get form component://c

Py4JJavaError: An error occurred while calling o180.getMetaForm.
: java.lang.NullPointerException
	at org.apache.ofbiz.widget.model.FormFactory.createModelForm(FormFactory.java:134)
	at org.apache.ofbiz.widget.model.FormFactory.createModelForm(FormFactory.java:130)
	at org.apache.ofbiz.widget.model.FormFactory.getFormFromLocation(FormFactory.java:71)
	at com.sagas.meta.FormManager.getModelForm(FormManager.java:79)
	at com.sagas.meta.FormManager.getMetaForm(FormManager.java:202)
	at com.sagas.meta.FormManager.getMetaForm(FormManager.java:177)
	at sun.reflect.GeneratedMethodAccessor48.invoke(Unknown Source)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.jav